<a href="https://colab.research.google.com/github/vikramkrishnan9885/MyColab/blob/master/BinaryImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Steps

1. Load Data
2. Preprocessing Data
3. Building the model
4. Defining the loss model
5. Defining optimizer
6. Train and evaluating the model
7. Deploying the model